#### Try to train a model to find the acupoint

In [2]:
import torch
import torchvision
from torchvision.models.detection import keypointrcnn_resnet50_fpn

In [3]:
model = keypointrcnn_resnet50_fpn(pretrained=True)

/home/robin/project/1122_CSIE/acupoint/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/robin/project/1122_CSIE/acupoint/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=KeypointRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=KeypointRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/keypointrcnn_resnet50_fpn_coco-fc266e95.pth" to /home/robin/.cache/torch/hub/checkpoints/keypointrcnn_resnet50_fpn_coco-fc266e95.pth
100%|██████████| 226M/226M [00:17<00:00, 13.2MB/s] 


In [65]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import json
import os

In [94]:
def _load_jsonl(file):
    data = []
    with open(file, "r") as f:
        for line in f.readlines():
            data.append(json.loads(line))
    return data
class AcupointDataset(Dataset):
    def __init__(self, data_path, idx, transform=None):
        self.data_path = f"{data_path}/imgs/{idx}"
        self.key_label = f"{data_path}/keypoints/{idx}.jsonl"
        self.bbox_label = f"{data_path}/labels/{idx}.jsonl"
        self.transform = transform

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx):
        img_path = f"{self.data_path}/{idx}.jpg"
        image = torchvision.io.read_image(img_path)
        image = torchvision.transforms.ToPILImage()(image)
        if self.transform:
            image = self.transform(image)
        keypoints = []
        data = _load_jsonl(self.key_label)
        for d in data:
            if d["image"] == img_path:
                keypoints = d["keypoints"]
                break
        data2 = _load_jsonl(self.bbox_label)
        x1, y1, x2, y2 = 0, 0, 0, 0
        for d in data2:
            if d["image"] == img_path:
                x1 = d['x1']
                y1 = d['y1']
                x2 = d['x2']
                y2 = d['y2']
                break

        targets = []
        for i in range(0, len(keypoints)):
            target = {}
            target["keypoints"] = torch.tensor(keypoints[i], dtype=torch.float32)
            target["labels"] = torch.tensor([1], dtype=torch.int64)
            target["boxes"] = torch.tensor([x1, y1, x2, y2], dtype=torch.float32)
            targets.append(target)
        return image, targets 

In [95]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
data = AcupointDataset("datasets/hand", 3115, transform)
data_loader = DataLoader(data, batch_size=1, shuffle=True)

In [96]:
images, _ = data[0]
print(images.shape)

torch.Size([3, 1920, 1080])


In [97]:
num_keypoints = 3
model.roi_heads.keypoint_predictor.kps_score_lowres = torch.nn.Conv2d(512, num_keypoints, 1)
model.roi_heads.keypoint_predictor.kps_score = torch.nn.Conv2d(256, num_keypoints, 1)

In [105]:
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

for epoch in range(epochs):
    for i, (images, targets) in enumerate(data_loader):
        images = images.to(device)
        new_targets = []
        for t in targets:
            t["keypoints"] = t["keypoints"].to(device)
            t["labels"] = t["labels"].to(device)
            t["boxes"] = t["boxes"].to(device)
            new_targets.append(t)

        optimizer.zero_grad()
        output = model(images, new_targets)
        predected_keypoints = output[0]["keypoints"]

        loss = loss_fn(predected_keypoints, targets)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Iteration {i}, Loss: {loss.item()}")


/home/robin/project/1122_CSIE/acupoint/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


RuntimeError: 0D or 1D target tensor expected, multi-target not supported